In [103]:
import pandas as pd
import os
from datetime import datetime
from datetime import date


In [104]:

allFiles = os.listdir('scrapedData')
csvFileNames = []
for fileName in allFiles:
    if "csvOut" in fileName:
        csvFileNames.append(fileName)
        

combined_csv = pd.concat( [ pd.read_csv("scrapedData\\" + f) for f in csvFileNames ] )        


In [112]:
pd.options.display.max_columns = None
#combined_csv = pd.read_csv("scrapedData\\" + csvFileNames[0])
numRows = len(combined_csv.index)
print(numRows)
columnCountCurrent = (len(combined_csv.index))
notAvailableColumn = ["#NA"] * numRows
newStringColumn = ["new"] * numRows


combined_csv['thumbnail_link'] = notAvailableColumn
combined_csv['comments_disabled'] = notAvailableColumn
combined_csv['ratings_disabled'] = notAvailableColumn
combined_csv['video_error_or_removed'] = notAvailableColumn

# Creating publishedDateCorrectFormat column
videoPublishTimeColumn = (combined_csv['videoPublishTime'])
videoPublishTimeColumnSplit1 = [i.split('T', 1)[0] for i in videoPublishTimeColumn]
videoPublishTimeStandard = [datetime.strptime(i, '%Y-%m-%d') for i in videoPublishTimeColumnSplit1]    # THis is in standard datetime object format
publishedDateCorrectFormat = [x.strftime("%d-%m-%y") for x in videoPublishTimeStandard]
combined_csv['publishedDateCorrectFormat'] = publishedDateCorrectFormat




# Creating trendingDateCorrectFormat
timeFetchedColumn = combined_csv['timeFetched']
timeFetchedColumnSplit1 = [i.split(' ', 1)[0] for i in timeFetchedColumn]
timeFetchedStandard = [datetime.strptime(i, '%Y-%m-%d') for i in timeFetchedColumnSplit1]
trendingDateCorrectFormat = [x.strftime("%d-%m-%y") for x in timeFetchedStandard]
combined_csv['trendingDateCorrectFormat'] = trendingDateCorrectFormat


# Calculating day difference
dayDifference = [(timeFetchedStandard[i] - videoPublishTimeStandard[i]).days for i in range(numRows)]
combined_csv['dayDifference'] = dayDifference

# newOrOldData column
combined_csv['newOrOldData'] = newStringColumn


#combined_csv.head()

10800


In [106]:
combined_csv.to_csv( "newDataOnly_csv_newFormat.csv", index=False )

In [107]:
oldData = pd.read_excel (r'finalOldData.xlsx')

In [113]:
combinedOldNew_csv = pd.concat( [oldData, combined_csv] )        
combinedOldNew_csv.to_csv( "oldAndNewData.csv", index=False )